## Download html files

request,bsでは動的コンテンツをとってこれないのでseleniumを使う

In [2]:
import requests

In [3]:
url="https://toho-vote.info/result"

In [4]:
r=requests.get(url)

In [5]:
r

<Response [200]>

In [6]:
r.content

b'<!DOCTYPE html>\n<html lang="ja">\n  <head>\n    <meta charset="UTF-8" />\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n\n    <link rel="icon" href="/favicon.ico" type="image/x-icon">\n    <link rel="icon" href="/icon.svg" type="image/svg+xml">\n    <link rel="apple-touch-icon" href="/apple-touch-icon.png">\n    <meta name="format-detection" content="telephone=no">\n\n    <title>\xe7\xac\xac18\xe5\x9b\x9e\xe6\x9d\xb1\xe6\x96\xb9Project\xe4\xba\xba\xe6\xb0\x97\xe6\x8a\x95\xe7\xa5\xa8</title>\n    <meta name="description" content="\xe6\x9d\xb1\xe6\x96\xb9Project\xe3\x81\xab\xe7\x99\xbb\xe5\xa0\xb4\xe3\x81\x99\xe3\x82\x8b\xe3\x82\xad\xe3\x83\xa3\xe3\x83\xa9\xe3\x82\xaf\xe3\x82\xbf\xe3\x83\xbc\xe3\x82\x84\xe9\x9f\xb3\xe6\xa5\xbd\xe3\x81\xaa\xe3\x81\xa9\xe3\x81\xae\xe4\xba\xba\xe6\xb0\x97\xe6\x8a\x95\xe7\xa5\xa8\xe3\x82\x92\xe8\xa1\x8c\xe3\x81\x86\xe9\x9d\x9e\xe5\x85\xac\xe5\xbc\x8f\xe3\x81\xae\xe3

In [ ]:
r.write()

## htmlからcsvファイル作成(scraping)

In [1]:
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

In [2]:
ddir="data"
filename="/18/第18回東方Project人気投票_人妖"+".html"

In [3]:
with open(ddir+filename,"r") as fp:
    htmltext=fp.read()
soup = BeautifulSoup(htmltext, "html.parser")

In [4]:
#document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.div

<div class="_title_en_1h8e9_51">Characters</div>

In [44]:
#document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
#soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div

<div class="_wrapper_1r0po_24"><header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header><div><div><p class="_count_1orw9_17">投票数 51,652票</p><table class="_result_1orw9_33"><thead><tr><th>順位</th><th class="_rank_prev_1orw9_133">前回</th><th class="_rank_prev2_1orw9_136">前々</th><th class="_name_1orw9_101">名前</th><th>ポイント</th><th>一推し</th><th>コメント</th><th>支援作品</th></tr></thead><tbody><tr><td class="_rank_1orw9_70"><div>1</div></td><td class="_rank_prev_1orw9_133">5</td><td class="_rank_prev2_1orw9_136">5</td><td class="_name_1orw9_101"><span>フランドール・スカーレット</span></td><td><strong>16,723</strong></td><td>3,730</td><td>3,633</td><td>19</td></tr><tr><td class="_rank_1orw9_70"><div>2</div></td><td class="_rank_prev_1orw9_133">2</td><td class="_rank_prev2_1orw9_136">4</td><td class="_name_1orw9_101"><span>霧雨 魔理沙</span></td><td><strong>16,578</strong></td><td>3,267</td><td>3,160</td><td>25</td></tr><tr><td class="_rank_1orw9_70"><div>3</div></td><td class=

In [5]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header

<header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header>

In [35]:
def get_tabledata(table):
#    for tr in table:        print(tr)
    return [[td.string for td in tr.find_all("td")] for tr in table]
#tr.find_all("td")

In [42]:
table_contents=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.find_all("tr")

In [43]:
table_contents

[<tr><th>順位</th><th class="_rank_prev_1orw9_133">前回</th><th class="_rank_prev2_1orw9_136">前々</th><th class="_name_1orw9_101">名前</th><th>ポイント</th><th>一推し</th><th>コメント</th><th>支援作品</th></tr>,
 <tr><td class="_rank_1orw9_70"><div>1</div></td><td class="_rank_prev_1orw9_133">5</td><td class="_rank_prev2_1orw9_136">5</td><td class="_name_1orw9_101"><span>フランドール・スカーレット</span></td><td><strong>16,723</strong></td><td>3,730</td><td>3,633</td><td>19</td></tr>,
 <tr><td class="_rank_1orw9_70"><div>2</div></td><td class="_rank_prev_1orw9_133">2</td><td class="_rank_prev2_1orw9_136">4</td><td class="_name_1orw9_101"><span>霧雨 魔理沙</span></td><td><strong>16,578</strong></td><td>3,267</td><td>3,160</td><td>25</td></tr>,
 <tr><td class="_rank_1orw9_70"><div>3</div></td><td class="_rank_prev_1orw9_133">3</td><td class="_rank_prev2_1orw9_136">2</td><td class="_name_1orw9_101"><span>博麗 霊夢</span></td><td><strong>15,679</strong></td><td>2,913</td><td>2,952</td><td>33</td></tr>,
 <tr><td class="_rank_1orw9_70

In [41]:
get_tabledata(table_contents)

[[],
 ['1', '5', '5', 'フランドール・スカーレット', '16,723', '3,730', '3,633', '19'],
 ['2', '2', '4', '霧雨 魔理沙', '16,578', '3,267', '3,160', '25'],
 ['3', '3', '2', '博麗 霊夢', '15,679', '2,913', '2,952', '33'],
 ['4', '1', '1', '魂魄 妖夢', '15,323', '3,791', '2,883', '29'],
 ['5', '6', '7', '十六夜 咲夜', '13,787', '2,493', '2,591', '9'],
 ['6', '4', '3', '古明地 こいし', '13,150', '2,473', '2,453', '48'],
 ['7', '7', '6', 'レミリア・スカーレット', '12,981', '2,215', '2,429', '8'],
 ['8', '8', '9', '藤原 妹紅', '8,728', '1,462', '1,542', '13'],
 ['9', '9', '8', '古明地 さとり', '7,600', '974', '1,310', '6'],
 ['10', '10', '13', '西行寺 幽々子', '6,960', '986', '1,257', '7'],
 ['11', '12', '11', 'アリス・マーガトロイド', '6,909', '1,036', '1,155', '18'],
 ['12', '11', '10', '射命丸 文', '6,607', '965', '1,065', '9'],
 ['13', '14', '14', '鈴仙・優曇華院・イナバ', '6,575', '843', '1,133', '10'],
 ['14', '13', '12', '東風谷 早苗', '6,185', '950', '1,046', '10'],
 ['15', '18', '24', 'チルノ', '5,850', '900', '1,284', '14'],
 ['16', '16', '15', '八雲 紫', '5,342', '643', '918', '7'